In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sympy as smp
from matplotlib import animation
from matplotlib.animation import PillowWriter

In [2]:
t, g, l1, l2, l3, l4, l5, r1, r2, r3, r4, r5, m1, m2, m3, m4, m5, i1, i2, i3, i4, i5 = smp.symbols('t g l_1 l_2 l_3 l_4 l_5 r_1 r_2 r_3 r_4 r_5 m_1 m_2 m_3 m_4 m_5 I_1 I_2 I_3 I_4 I_5')
q1, q2, q3, q4, q5, a = smp.symbols(r'\theta_1 \theta_2 \theta_3 \theta_4 \theta_5 \alpha', cls=smp.Function)

In [3]:
a

\alpha

In [4]:
# t, g, l1, l2, m1, m2, m3, k, L0 = smp.symbols('t g l_1 l_2 m_1 m_2 m_3 k L_0')
# the1, the2 = smp.symbols(r'\theta_1 \theta_2', cls=smp.Function)

In [5]:
q1 = q1(t)
q2 = q2(t)
q3 = q3(t)
q4 = q4(t)
q5 = q5(t)
a = -90 + q5 + q2 + q3

q1_d = smp.diff(q1, t)
q2_d = smp.diff(q2, t)
q3_d = smp.diff(q3, t)
q4_d = smp.diff(q4, t)
q5_d = smp.diff(q5, t)
q1_dd = smp.diff(q1_d, t)
q2_dd = smp.diff(q2_d, t)
q3_dd = smp.diff(q3_d, t)
q4_dd = smp.diff(q4_d, t)
q5_dd = smp.diff(q5_d, t)

In [6]:
smp.Matrix([[l1-r1],[0]])

Matrix([
[l_1 - r_1],
[        0]])

In [7]:
a

\theta_2(t) + \theta_3(t) + \theta_5(t) - 90

In [8]:
def get_rot_mat(theta):
    cth = smp.cos(theta)
    sth = smp.sin(theta)
    rot_mat = smp.Matrix([[cth, -1*sth], [sth, cth]])
    return rot_mat

In [9]:
locations = [l1-r1, l2-r2, l3-r3, l4-r4, l5-r5]

array = smp.Matrix([[locations[3]],[0]])
A4 = get_rot_mat(q4)
# rot_mat = smp.Matrix([[smp.cos(q1), -1*smp.sin(q1)], [smp.sin(q1), smp.cos(q1)]])

In [10]:
A0 = get_rot_mat(a)
A1 = get_rot_mat(q1)
A2 = get_rot_mat(q2)
A3 = get_rot_mat(q3)
A4 = get_rot_mat(q4)
A5 = get_rot_mat(q5)


In [11]:
def get_key(dict, where):
    for key, value in dict.items():
        if value == where:
            return key
    return 0

In [12]:
def get_parts(where, reverse = False):
    parts = {'tibia1': 1, 'femur1': 2, 'torso': 3, 'femur2': 4, 'tibia2': 5}
    part_maping = {0: 0, 1: 3, 2: 2, 3: 5, 4: 1, 5: 4} 
    
    if reverse:
        index_part_maping = get_key(part_maping, where)
        part_name = get_key(parts, index_part_maping)
        return part_name
    
    return parts[where]

In [13]:
def get_path(where):
    
    parts = {'tibia1': 1, 'femur1': 2, 'torso': 3, 'femur2': 4, 'tibia2': 5}
    part = parts[where]
    path = []

    for i in range(part, 0, -1):
        if part > 3:
            if i != 3:
                path.append(i)
        else:
            path.append(i)
            
    return path

In [14]:
def get_locations(path_part_map, cg_or_not = False):
    locations = smp.Matrix([l1, l2, l3, l4, l5])
    locations_cg = smp.Matrix([r1, l2-r2, l3-r3, r4, r5])
    if cg_or_not:
        locations[path_part_map[0]-1] = locations_cg[path_part_map[0]-1]
    return locations

In [15]:
def get_path_part_map(where, cg_or_not = False):
    
    path = get_path(where)
    part_maping = {0: 0, 1: 3, 2: 2, 3: 5, 4: 1, 5: 4} 
    path_part_map = [part_maping[i] for i in path]
    return path_part_map

In [16]:
# # path_part_map.reverse()
# path_part_map = get_path_part_map('torso')
# locations = get_locations(path_part_map, True)
# for i in range(len(path)):
#     link = path_part_map[len(path)-i-1]-1
#     print(smp.Matrix([[locations[link]],[0]]))
# print(path_part_map)


In [17]:
def get_coord(where, cg_or_not = False):
    
    parts = {'tibia1': 1, 'femur1': 2, 'torso': 3, 'femur2': 4, 'tibia2': 5}
    part_maping = {0: 0, 1: 3, 2: 2, 3: 5, 4: 1, 5: 4} 
    part = parts[where]
    
    path = get_path(where)
    path_part_map = get_path_part_map(where)
    property_maps = {0: [0, A0],1: [q1, A1], 2: [q2, A2], 3: [q3, A3], 4: [q4, A4], 5: [q5, A5]}
    
    #tibia1 = q3,l3,r3
    #femur1 = q2,l2,r2...
    
    locations = get_locations(path_part_map, cg_or_not)
    
    tibia_1 = smp.Matrix([[locations[2]],[0]])
#     femur_1 = smp.Matrix([[locations[1]],[0]])
#     tibia_2 = smp.Matrix([[locations[0]],[0]])
#     femur_2 = smp.Matrix([[locations[3]],[0]])
#     torso = smp.Matrix([[locations[4]],[0]])
    
    rot_mat_product = 1 #product of rotational matrices
    
    coordinates = smp.Matrix([[0],[0]])
    
    for i in range(len(path)):
        rot_mat_product = rot_mat_product*property_maps[part_maping[i]][1]
        link = path_part_map[len(path)-i-1]-1
        coordinates += rot_mat_product*smp.Matrix([[locations[link]],[0]])
    rot_mat_product *= property_maps[path_part_map[0]][1]
    coordinates += rot_mat_product*smp.Matrix([[0],[0]])
    
    return coordinates

In [18]:
# coord = smp.simplify(smp.Matrix(get_coord('torso', True)))
# coord[1]

In [19]:
def get_velocity_squared(coord):
    x = coord[0]
    y = coord[1]
    
    x_d = smp.diff(coord[0], t)
    y_d = smp.diff(coord[0], t)
    
    v = x_d**2 + y_d**2
    
    return v

In [20]:
def get_all_coords():
    parts = {'tibia1': 1, 'femur1': 2, 'torso': 3, 'femur2': 4, 'tibia2': 5}
    part_maping = {0: 0, 1: 3, 2: 2, 3: 5, 4: 1, 5: 4} 
    
    coords = {1: get_coord('tibia1', True), 2: get_coord('femur1', True), 3: get_coord('torso', True), 4: get_coord('femur2', True), 5: get_coord('tibia2', True)}
    return coords

In [22]:
coords = get_all_coords()

{1: Matrix([
 [(l_3 - r_3)*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)],
 [(l_3 - r_3)*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)]]),
 2: Matrix([
 [l_3*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) + (l_2 - r_2)*(-sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*sin(\theta_3(t)) + cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)))],
 [ l_3*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) + (l_2 - r_2)*(sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)) + sin(\theta_3(t))*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90))]]),
 3: Matrix([
 [l_2*(-sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*sin(\theta_3(t)) + cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t))) + l_3*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) + r_5*((-sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*sin(\theta_3(t)) + cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)))*cos(\theta_2(t)) + (-sin(\theta_2(t) + \theta_3(t) + \th

In [29]:
def get_T_and_V():
    
    T = 0
    V = 0
    energy_properties = {1: [m1, i1, q1_d], 2: [m2, i2, q2_d], 3: [m3, i3, q3_d], 4: [m4, i4, q4_d], 5: [m5, i5, q5_d]}
    all_coords = get_all_coords()
    
    for i in range(1, len(energy_properties)+1):
        coord = all_coords[i]
        T += smp.Rational(1,2) * energy_properties[i][0] * get_velocity_squared(coord)
        T += smp.Rational(1,2) * energy_properties[i][1] * (energy_properties[i][2]**2)
        V += energy_properties[i][0] * g * coord[1]
        
    return [T,V]

In [30]:
smp.Matrix(get_T_and_V())

Matrix([
[I_1*Derivative(\theta_1(t), t)**2/2 + I_2*Derivative(\theta_2(t), t)**2/2 + I_3*Derivative(\theta_3(t), t)**2/2 + I_4*Derivative(\theta_4(t), t)**2/2 + I_5*Derivative(\theta_5(t), t)**2/2 + m_1*(l_3 - r_3)**2*(Derivative(\theta_2(t), t) + Derivative(\theta_3(t), t) + Derivative(\theta_5(t), t))**2*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)**2 + m_2*(-l_3*(Derivative(\theta_2(t), t) + Derivative(\theta_3(t), t) + Derivative(\theta_5(t), t))*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) + (l_2 - r_2)*(-(Derivative(\theta_2(t), t) + Derivative(\theta_3(t), t) + Derivative(\theta_5(t), t))*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)) - (Derivative(\theta_2(t), t) + Derivative(\theta_3(t), t) + Derivative(\theta_5(t), t))*sin(\theta_3(t))*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) - sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t))*Derivative(\theta_3(t), t) - sin(\theta_3(t))*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90

In [34]:
def get_lagrange():
    
    Energies = get_T_and_V()
    
    T = Energies[0]
    V = Energies[1]
    
    return T-V

In [38]:
L = get_lagrange()

In [39]:
L

I_1*Derivative(\theta_1(t), t)**2/2 + I_2*Derivative(\theta_2(t), t)**2/2 + I_3*Derivative(\theta_3(t), t)**2/2 + I_4*Derivative(\theta_4(t), t)**2/2 + I_5*Derivative(\theta_5(t), t)**2/2 - g*m_1*(l_3 - r_3)*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) - g*m_2*(l_3*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) + (l_2 - r_2)*(sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)) + sin(\theta_3(t))*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90))) - g*m_3*(l_2*(sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)) + sin(\theta_3(t))*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)) + l_3*sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90) + r_5*((-sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*sin(\theta_3(t)) + cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)))*sin(\theta_2(t)) + (sin(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90)*cos(\theta_3(t)) + sin(\theta_3(t))*cos(\theta_2(t) + \theta_3(t) + \theta_5(t) - 90))*cos(\theta_2(t))

In [ ]:
LE1 = smp.diff(L, q1) - smp.diff(smp.diff(L, q1_d), t).simplify()
print('Done')
LE1
LE2 = smp.diff(L, q2) - smp.diff(smp.diff(L, q2_d), t)
print('Dome')
LE2.simplify()

Done
Dome


In [ ]:

LE2 = smp.diff(L, q2) - smp.diff(smp.diff(L, q2_d), t).simplify()
print('Done')
LE3 = smp.diff(L, q3) - smp.diff(smp.diff(L, q3_d), t).simplify()
print('Done')
LE4 = smp.diff(L, q4) - smp.diff(smp.diff(L, q4_d), t).simplify()
print('Done')
LE5 = smp.diff(L, q5) - smp.diff(smp.diff(L, q5_d), t).simplify()
print('Done')